## MLFlow model 

In this code section we will cover how to create refractored code to automate the creation of numeroous models within mlflow. We will cover how to log your model, its parameters and metrics, and how to monitor this in MLFlow 

In [1]:
import os
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from mlflow.models import infer_signature
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec

# --- Constants ---
N_SPLITS_CV = 5
RANDOM_SEED = 42
MLFLOW_EXPERIMENT_NAME = "Telco Churn Prediction CV & Model Logging"

# --- Data Loading and Initial Preprocessing ---
def load_data(data_path: str):
    """
    Loads the Telco Customer Churn dataset, performs initial cleaning,
    and separates features (X) from the target (y).
    """
    churn = pd.read_csv(data_path)
    churn['TotalCharges'] = pd.to_numeric(churn['TotalCharges'], errors='coerce')
    churn.dropna(subset=['TotalCharges'], inplace=True)
    
    y = LabelEncoder().fit_transform(churn['Churn'])
    X = churn.drop('Churn', axis=1)
    
    return X, y

# --- Preprocessing Pipeline Definition ---
def define_preprocessor(X: pd.DataFrame):
    """
    Defines the preprocessing steps for numerical and categorical features.
    """
    # Identify categorical and numerical features
    categorical_features = X.select_dtypes(include=['object']).columns
    numerical_features = X.select_dtypes(include=np.number).columns

    # Create preprocessing pipelines for numerical and categorical features
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Create a preprocessor using ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        remainder='passthrough'  # Keep other columns not specified
    )
    return preprocessor

# --- Model Training, Evaluation, and MLflow Logging Function ---
def train_evaluate_log_model(model_name: str, classifier, X_full: pd.DataFrame, y_full: np.ndarray, model_params: dict, preprocessor: ColumnTransformer, n_splits_cv_param: int):
    """
    Trains and evaluates a given model using cross-validation, logs metrics and the model to MLflow.
    """
    with mlflow.start_run(run_name=f"{model_name} Training"):
        print(f"\n--- Training {model_name} with Cross-Validation ---")
        mlflow.log_params(model_params)

        pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('classifier', classifier(**model_params))])

        cv_accuracies = []
        cv_precisions = []
        cv_recalls = []
        cv_f1s = []
        cv_roc_aucs = []

        skf = StratifiedKFold(n_splits=n_splits_cv_param, shuffle=True, random_state=RANDOM_SEED)

        for fold, (train_index, val_index) in enumerate(skf.split(X_full, y_full)):
            X_train_fold, X_val_fold = X_full.iloc[train_index], X_full.iloc[val_index]
            y_train_fold, y_val_fold = y_full[train_index], y_full[val_index]

            pipeline.fit(X_train_fold, y_train_fold)
            y_pred = pipeline.predict(X_val_fold)
            y_proba = pipeline.predict_proba(X_val_fold)[:, 1] if hasattr(pipeline, "predict_proba") else [0] * len(y_pred)

            accuracy = accuracy_score(y_val_fold, y_pred)
            precision = precision_score(y_val_fold, y_pred, average='weighted', zero_division=0)
            recall = recall_score(y_val_fold, y_pred, average='weighted')
            f1 = f1_score(y_val_fold, y_pred, average='weighted')
            roc_auc = roc_auc_score(y_val_fold, y_proba)

            cv_accuracies.append(accuracy)
            cv_precisions.append(precision)
            cv_recalls.append(recall)
            cv_f1s.append(f1)
            cv_roc_aucs.append(roc_auc)

            mlflow.log_metrics({
                f"fold_{fold+1}_accuracy": accuracy,
                f"fold_{fold+1}_precision": precision,
                f"fold_{fold+1}_recall": recall,
                f"fold_{fold+1}_f1": f1,
                f"fold_{fold+1}_roc_auc": roc_auc
            })

        mean_accuracy = np.mean(cv_accuracies)
        mean_precision = np.mean(cv_precisions)
        mean_recall = np.mean(cv_recalls)
        mean_f1 = np.mean(cv_f1s)
        mean_roc_auc = np.mean(cv_roc_aucs)

        print(f"  {model_name} CV Results:")
        print(f"    Mean Test accuracy: {mean_accuracy:.4f} (Std: {np.std(cv_accuracies):.4f})")
        print(f"    Mean Test precision_weighted: {mean_precision:.4f} (Std: {np.std(cv_precisions):.4f})")
        print(f"    Mean Test recall_weighted: {mean_recall:.4f} (Std: {np.std(cv_recalls):.4f})")
        print(f"    Mean Test f1_weighted: {mean_f1:.4f} (Std: {np.std(cv_f1s):.4f})")
        print(f"    Mean Test ROC AUC: {mean_roc_auc:.4f} (Std: {np.std(cv_roc_aucs):.4f})")

        mlflow.log_metrics({
            "mean_cv_accuracy": mean_accuracy,
            "mean_cv_precision": mean_precision,
            "mean_cv_recall": mean_recall,
            "mean_cv_f1": mean_f1,
            "mean_cv_roc_auc": mean_roc_auc
        })

        print(f"  Fitting final {model_name} pipeline on full training data for logging...")
        full_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                         ('classifier', classifier(**model_params))])
        full_pipeline.fit(X_full, y_full)

        # Infer signature for MLflow logging
        example_input = X_full.sample(1, random_state=RANDOM_SEED)
        signature = infer_signature(example_input, full_pipeline.predict(example_input))

        # Log the final trained pipeline
        mlflow.sklearn.log_model(
            sk_model=full_pipeline, 
            artifact_path="final_model_pipeline",
            signature=signature,
            input_example=example_input
        )
        print(f"  Final {model_name} pipeline logged to: {mlflow.active_run().info.artifact_uri}/final_model_pipeline")

# --- Main Execution Function ---
def main():
    """
    Orchestrates the entire ML pipeline: data loading, preprocessing,
    and training/logging multiple models.
    """
    print("Starting Telco Churn Classification with CV and Model Logging...")

    # Set MLflow experiment
    mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

    # Load data
    data_path = os.path.join(os.getcwd(), 'raw', 'WA_Fn-UseC_-Telco-Customer-Churn.csv')
    X_full, y_full = load_data(data_path)
    print(f"Data loaded. X_full shape: {X_full.shape}, y_full shape: {y_full.shape}")

    # Define preprocessor
    preprocessor = define_preprocessor(X_full)
    print("Preprocessor defined.")

    # Define models to compare
    models_to_compare = [
        {
            "name": "Logistic Regression",
            "classifier": LogisticRegression,
            "params": {"solver": "liblinear", "random_state": RANDOM_SEED}
        },
        {
            "name": "Random Forest Classifier",
            "classifier": RandomForestClassifier,
            "params": {"n_estimators": 100, "random_state": RANDOM_SEED}
        },
        {
            "name": "Gradient Boosting Classifier",
            "classifier": GradientBoostingClassifier,
            "params": {"n_estimators": 100, "random_state": RANDOM_SEED}
        }
    ]

    # Run the full pipeline for each model
    for model_info in models_to_compare:
        train_evaluate_log_model(
            model_info["name"],
            model_info["classifier"],
            X_full,
            y_full,
            model_info["params"],
            preprocessor,  # Pass the preprocessor defined in main()
            N_SPLITS_CV    # Pass the N_SPLITS_CV constant
        )

    print("\nAll Telco Churn models processed with CV and models logged to MLflow.")
    print(f"To view results, run 'mlflow ui' in your terminal and navigate to the '{MLFLOW_EXPERIMENT_NAME}' experiment.")

# --- Guard to run main() when script is executed directly ---
if __name__ == "__main__":
    main()

Starting Telco Churn Classification with CV and Model Logging...
Data loaded. X_full shape: (7032, 20), y_full shape: (7032,)
Preprocessor defined.

--- Training Logistic Regression with Cross-Validation ---
  Logistic Regression CV Results:
    Mean Test accuracy: 0.8039 (Std: 0.0056)
    Mean Test precision_weighted: 0.7959 (Std: 0.0056)
    Mean Test recall_weighted: 0.8039 (Std: 0.0056)
    Mean Test f1_weighted: 0.7982 (Std: 0.0053)
    Mean Test ROC AUC: 0.8449 (Std: 0.0026)
  Fitting final Logistic Regression pipeline on full training data for logging...


C:\Users\hanam\AppData\Roaming\Python\Python312\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/07/31 15:07:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


2025/07/31 15:07:48 INFO mlflow.models.model: Found the following environment variables used during model inference: [OPENAI_API_KEY]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


  Final Logistic Regression pipeline logged to: file:///c:/Users/hanam/OneDrive/repos/BA-MLOps/03%20Deploying%20%26%20Productionising%20ML%20Models/Example%20Exercises/mlruns/428641126348462805/ec33ffcbb3534082b702d12368190902/artifacts/final_model_pipeline

--- Training Random Forest Classifier with Cross-Validation ---
  Random Forest Classifier CV Results:
    Mean Test accuracy: 0.7948 (Std: 0.0044)
    Mean Test precision_weighted: 0.7820 (Std: 0.0050)
    Mean Test recall_weighted: 0.7948 (Std: 0.0044)
    Mean Test f1_weighted: 0.7817 (Std: 0.0037)
    Mean Test ROC AUC: 0.8284 (Std: 0.0032)
  Fitting final Random Forest Classifier pipeline on full training data for logging...


C:\Users\hanam\AppData\Roaming\Python\Python312\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/07/31 15:09:19 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


  Final Random Forest Classifier pipeline logged to: file:///c:/Users/hanam/OneDrive/repos/BA-MLOps/03%20Deploying%20%26%20Productionising%20ML%20Models/Example%20Exercises/mlruns/428641126348462805/d253d2f787b64790b8d9badf53184caa/artifacts/final_model_pipeline

--- Training Gradient Boosting Classifier with Cross-Validation ---
  Gradient Boosting Classifier CV Results:
    Mean Test accuracy: 0.8039 (Std: 0.0065)
    Mean Test precision_weighted: 0.7935 (Std: 0.0069)
    Mean Test recall_weighted: 0.8039 (Std: 0.0065)
    Mean Test f1_weighted: 0.7947 (Std: 0.0061)
    Mean Test ROC AUC: 0.8473 (Std: 0.0047)
  Fitting final Gradient Boosting Classifier pipeline on full training data for logging...


C:\Users\hanam\AppData\Roaming\Python\Python312\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/07/31 15:09:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


  Final Gradient Boosting Classifier pipeline logged to: file:///c:/Users/hanam/OneDrive/repos/BA-MLOps/03%20Deploying%20%26%20Productionising%20ML%20Models/Example%20Exercises/mlruns/428641126348462805/852abb0705264e628c60a6fda4feb828/artifacts/final_model_pipeline

All Telco Churn models processed with CV and models logged to MLflow.
To view results, run 'mlflow ui' in your terminal and navigate to the 'Telco Churn Prediction CV & Model Logging' experiment.


## Single and batch prediction locally

In [4]:
import mlflow
import pandas as pd
from pydantic import BaseModel
from typing import List, Dict, Any

# --- Configuration ---
# IMPORTANT: Replace this with the actual MLflow URI of your model
# You can find this in the MLflow UI or the output of your logging cell
# For example: "runs:/<YOUR_RUN_ID>/final_model_pipeline" or "file:///path/to/your/mlruns/run_id/artifacts/final_model_pipeline"
MLFLOW_MODEL_URI = r"file///C:/Users/hanam/OneDrive/repos/BA-MLOps/03 Deploying & Productionising ML Models/Example Exercises/mlruns/428641126348462805/models/m-47434a991eb2402bae2a34fdbc097aee/artifacts"

# --- Define Pydantic Model for Input Data ---
# IMPORTANT: Adjust these features and types to precisely match your model's expected input
# These are common features for a Telco Churn dataset.
class TelcoChurnInput(BaseModel):
    customerID: str
    gender: str
    SeniorCitizen: int
    Partner: str
    Dependents: str
    tenure: int
    PhoneService: str
    MultipleLines: str
    InternetService: str
    OnlineSecurity: str
    OnlineBackup: str
    DeviceProtection: str
    TechSupport: str
    StreamingTV: str
    StreamingMovies: str
    Contract: str
    PaperlessBilling: str
    PaymentMethod: str
    MonthlyCharges: float
    TotalCharges: float

# --- Load MLflow Model ---
try:
    loaded_model = mlflow.pyfunc.load_model(MLFLOW_MODEL_URI)
    print(f"MLflow model loaded successfully from {MLFLOW_MODEL_URI}")
except Exception as e:
    print(f"Error loading MLflow model: {e}")
    loaded_model = None # Set to None to handle cases where model loading fails

# --- Prediction Functions ---
def predict_single_entry(data: TelcoChurnInput) -> Dict[str, Any]:
    """
    Performs prediction for a single customer entry.
    """
    if loaded_model is None:
        return {"error": "Model not loaded. Cannot perform prediction."}

    # Convert Pydantic model to a pandas DataFrame
    input_df = pd.DataFrame([data.model_dump()]) # Changed .dict() to .model_dump()

    # Handle 'TotalCharges' being potentially empty string
    if 'TotalCharges' in input_df.columns:
        input_df['TotalCharges'] = pd.to_numeric(input_df['TotalCharges'], errors='coerce')
        input_df['TotalCharges'] = input_df['TotalCharges'].fillna(0) # Or another suitable imputation

    predictions = loaded_model.predict(input_df)
    # Assuming your model outputs a single prediction (e.g., 0 or 1)
    # If your model outputs probabilities, adjust this.
    prediction_result = predictions[0]

    return {"prediction": int(prediction_result)} # Convert to int for JSON serialization

def predict_batch_entry(data_list: List[TelcoChurnInput]) -> List[Dict[str, Any]]:
    """
    Performs batch predictions for multiple customer entries.
    """
    if loaded_model is None:
        return [{"error": "Model not loaded. Cannot perform prediction."}]

    # Convert list of Pydantic models to a pandas DataFrame
    input_df = pd.DataFrame([data.model_dump() for data in data_list]) # Changed .dict() to .model_dump()

    # Handle 'TotalCharges' being potentially empty string
    if 'TotalCharges' in input_df.columns:
        input_df['TotalCharges'] = pd.to_numeric(input_df['TotalCharges'], errors='coerce')
        input_df['TotalCharges'] = input_df['TotalCharges'].fillna(0) # Or another suitable imputation

    predictions = loaded_model.predict(input_df)

    results = [{"prediction": int(pred)} for pred in predictions]
    return results

# --- Example Usage (within Jupyter Notebook) ---
if __name__ == "__main__":
    if loaded_model:
        # Single entry example
        single_customer_data = TelcoChurnInput(
            customerID="1234-ABCD", gender="Male", SeniorCitizen=0, Partner="Yes",
            Dependents="No", tenure=24, PhoneService="Yes", MultipleLines="No",
            InternetService="Fiber optic", OnlineSecurity="No", OnlineBackup="Yes",
            DeviceProtection="No", TechSupport="No", StreamingTV="Yes",
            StreamingMovies="Yes", Contract="Month-to-month", PaperlessBilling="Yes",
            PaymentMethod="Electronic check", MonthlyCharges=85.0, TotalCharges=2040.0
        )
        print("\n--- Single Entry Prediction ---")
        single_prediction = predict_single_entry(single_customer_data)
        print(f"Prediction for single entry: {single_prediction}")

        # Batch entry example
        batch_customers_data = [
            TelcoChurnInput(
                customerID="1234-ABCD", gender="Male", SeniorCitizen=0, Partner="Yes",
                Dependents="No", tenure=24, PhoneService="Yes", MultipleLines="No",
                InternetService="Fiber optic", OnlineSecurity="No", OnlineBackup="Yes",
                DeviceProtection="No", TechSupport="No", StreamingTV="Yes",
                StreamingMovies="Yes", Contract="Month-to-month", PaperlessBilling="Yes",
                PaymentMethod="Electronic check", MonthlyCharges=85.0, TotalCharges=2040.0
            ),
            TelcoChurnInput(
                customerID="5678-EFGH", gender="Female", SeniorCitizen=1, Partner="No",
                Dependents="No", tenure=1, PhoneService="Yes", MultipleLines="No",
                InternetService="DSL", OnlineSecurity="No", OnlineBackup="No",
                DeviceProtection="No", TechSupport="No", StreamingTV="No",
                StreamingMovies="No", Contract="Month-to-month", PaperlessBilling="No",
                PaymentMethod="Mailed check", MonthlyCharges=29.85, TotalCharges=29.85
            )
        ]
        print("\n--- Batch Entry Prediction ---")
        batch_predictions = predict_batch_entry(batch_customers_data)
        print(f"Predictions for batch entries: {batch_predictions}")
    else:
        print("Model was not loaded, cannot run examples.")

MLflow model loaded successfully from file///C:/Users/hanam/OneDrive/repos/BA-MLOps/03 Deploying & Productionising ML Models/Example Exercises/mlruns/428641126348462805/models/m-47434a991eb2402bae2a34fdbc097aee/artifacts

--- Single Entry Prediction ---
Prediction for single entry: {'prediction': 0}

--- Batch Entry Prediction ---
Predictions for batch entries: [{'prediction': 0}, {'prediction': 1}]
